# Notebook which visualises original trajectories and compares them to disk-hashed trajectories


In [9]:
import folium
import os
import sys
currentdir = os.path.dirname(os.path.abspath("__file__"))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from hashed_similarities.disk_similarity import (
    generate_disk_hash_similarity_coordinates,
)
# Rome coordinates
R_MAX_LON = 12.53
R_MIN_LON = 12.44
R_MAX_LAT = 41.93
R_MIN_LAT = 41.88


ROME_META_FILE = "../dataset/rome/output/META-50.txt"

In [10]:
# Initialize a Folium map at a central point
map = folium.Map(location=[41.9, 12.5], zoom_start=12)

folium.Marker([R_MAX_LAT, R_MAX_LON]).add_to(map)
folium.Marker([R_MIN_LAT, R_MIN_LON]).add_to(map)

In [11]:
def read_coordinates(file_path):
    coordinates = []
    with open("../dataset/rome/output/" + file_path, "r") as file:
        count = 0
        for line in file:
            count += 1
            lat, lon = line.strip().split(", ")
            coordinates.append((float(lat), float(lon)))
        print(f"Number of coordinates for {file}: {str(count)}")
    return coordinates


# Read the index file to get the list of files containing coordinates
with open(ROME_META_FILE, "r") as index_file:
    for file_name in index_file:
        file_name = file_name.strip()  # Remove newline characters

        coordinates = read_coordinates(file_name)
        # Plot each coordinate on the map
        for lat, lon in coordinates:
            folium.CircleMarker(
                [lat, lon], radius=5, color="red", fill=True, fill_color="red"
            ).add_to(map)
        folium.PolyLine(coordinates, color="red",
                        weight=4, opacity=1).add_to(map)


# Save the map to an HTML file

output_file_path = os.path.join("output/", "rome_map.html")

map.save(output_file_path)

Number of coordinates for <_io.TextIOWrapper name='../dataset/rome/output/R_ATG.txt' mode='r' encoding='UTF-8'>: 194


In [12]:
measure = "dtw"
disk_coordinates, disks = generate_disk_hash_similarity_coordinates(
    city="rome", diameter=2.2, layers=1, disks=50, measure=measure, size=50
)

print("Number of layers", len(disks))
for layer in disks:
    print(f"Number of disks in layer {layer}: {len(disks[layer])}")
    for disk in disks[layer]:
        lat, lon = disk
        # print(f"Disk \t{disk}")
        folium.CircleMarker(
            [lat, lon], radius=3, color="green", fill=True, fill_color="green"
        ).add_to(map)

keyCount = 0
for key, disks in disk_coordinates.items():
    layerCount = 0
    print("Number of disks intersected for ", key, ": ", len(disks))
    for disk in disks:
        lat, lon = disk
        folium.CircleMarker(
            [lat, lon], radius=5, color="blue", fill=True, fill_color="blue"
        ).add_to(map)
    # folium.PolyLine(disks, color="blue",
    #                 weight=4, opacity=1).add_to(map)

output_file_path = os.path.join("output/", "rome_hashed_and_original.html")

map.save(output_file_path)

Number of layers 1
Number of disks in layer 0: 50
Number of disks intersected for  R_ATG :  11
